## MSMOTE
http://sherry-data.tistory.com/22

https://datascienceschool.net/view-notebook/c1a8dad913f74811ae8eef5d3bedc0c3/

In [20]:
from imblearn.over_sampling import *
import pandas as pd

## 1. 언더샘플링하지 않은 전체 데이터 가져오기

In [4]:
# 구분 최초 설정 필요

df = 'all'
target_temp_df = 'N'  # 정상:부실의 데이터 비율을 맞추기 위한 데이터셋을 사용할 건지 여부 (데이터스케일링 쥬피터 결과 기준)

# main/all/core 3가지로 설정 


# core설정시 확인해야 함
# R을 통해 확인한 stepwise 유의 변수 리스트
core_factors = ['DR00000136','6000201001O0','6000903016D1','FNMKFN02','6000901002D1','S41000210FD1',
'6000207003O0','DR00000052','6000906001D6','DR00000156','6000901001D3','DR00000082',
'S41000210FD2','6000902001D2','6000908001D3','6000904001D3','6000908001D2','S41B0D1009O0',
'6000901002D3','6000903001D2','6000403001O0','CO10100170O0','DR00000113','6000908001D7']

#  키별로 비율 배분이 잘 됐는지 확인
if df == 'all':
    max_idx = 707
else:
    max_idx = 123
    

### 1). 데이터셋 셋팅

In [12]:
import pickle # 파이썬 객체저장을 위한 모듈

# 파이썬 객체 읽어오기

if target_temp_df != 'Y':
    if df == 'all':
        with open('./pickles/dataset_all.p', 'rb') as file:    # hello.txt 파일을 바이너리 읽기 모드(rb)로 열기
            train_set = pickle.load(file)
            test_set = pickle.load(file)
    else:
        with open('./pickles/dataset_main.p', 'rb') as file:    # hello.txt 파일을 바이너리 읽기 모드(rb)로 열기
            train_set = pickle.load(file)
            test_set = pickle.load(file)
else:
# 정상:부실의 데이터 비율 조정 기준
    if df == 'all':
        with open('./pickles/data6vs4_all.p', 'rb') as file:    # hello.txt 파일을 바이너리 읽기 모드(rb)로 열기
            temp_train_set = pickle.load(file)
            temp_test_set = pickle.load(file)
            af_cols = pickle.load(file)
    else:
        with open('./pickles/data6vs4_main.p', 'rb') as file:    # hello.txt 파일을 바이너리 읽기 모드(rb)로 열기
            temp_train_set = pickle.load(file)
            temp_test_set = pickle.load(file)
            af_cols = pickle.load(file)
            
# 필요없는 열 삭제
if target_temp_df == 'Y':
    train_set = temp_train_set.drop(['index', 0, max_idx-1,max_idx],axis=1)  # 열삭제
    test_set = temp_test_set.drop(['index', 0, max_idx-1,max_idx],axis=1)  # 열삭제


# 항목명 (열명) 셋팅
if target_temp_df == 'Y':
    factor_cols = list(af_cols[1:-2])
    train_set.columns = factor_cols
    test_set.columns = factor_cols


In [13]:
# core인 경우 main에서 핵심 변수만 추림

if df == 'core':
    final_factors = ['key', 'industry', 'label']
    final_factors = final_factors + core_factors
    train_set = train_set[final_factors]
    test_set = test_set[final_factors]
else:
    pass

In [14]:
df, train_set.shape, test_set.shape

('all', (5290, 705), (2336, 705))

### 2) 데이터 분리 및 타입변경

In [21]:
# 데이터 분리
cols = train_set.columns.values
train_info = train_set[cols[0:3]]
X_train = train_set[cols[3:]]
Y_train = train_set['label']
train_len = X_train.shape[0]

cols = test_set.columns.values
test_info = test_set[cols[0:3]]
X_test = test_set[cols[3:]]
Y_test = test_set['label']
test_len = X_test.shape[0]

In [16]:
X_train.shape, Y_train.shape

((5290, 702), (5290,))

## 2. 분리전 전처리된 데이터 가져오기

In [22]:
if df == 'all':
    address = './datas/raw_all_preprocessing.csv'
else:
    address = './datas/raw_main_preprocessing.csv'

data = pd.read_csv(address, header=0)  
cols = data.columns.values
cols = cols[1:].tolist()  # 주의! 


In [23]:
data.shape

(7626, 706)

In [24]:
data.head()

,Unnamed: 0,key,industry,label,S430000600O0,3001104004O0,3001212005O0,3001403004O0,3001111005O0,S430000100O0,...,DR00000157,DR00000158,DR00000159,DR00000160,DR00000161,DR00000162,DR00000163,DR00000164,DR00000165,DR00000166
0,0,2013000020,C,0,11.716837,2.969817,0.802569,0.416889,-1.886927,2.225912,...,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2013000040,C,0,10.864216,2.969817,2.881315,2.845274,-1.886927,2.225912,...,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2013000050,C,0,12.607617,2.969817,2.881315,2.845274,-0.363248,-0.851603,...,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2013000070,M,0,13.337241,2.969817,2.881315,2.845274,-1.886927,-0.851603,...,3.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,2013000080,C,0,14.263528,2.969817,2.881315,0.095426,-0.456624,-0.851603,...,3.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [18]:
from collections import Counter
from sklearn.svm import SVC

x1_min = X00[:, 0].min() - 2
x1_max = X00[:, 0].max() + 2
x2_min = X00[:, 1].min() - 2
x2_max = X00[:, 1].max() + 2

def plot_samples(X=None, y=None):
    XX, YY = np.mgrid[x1_min:x1_max:300j, x2_min:x2_max:300j]
    if X is None:
        X = X0
        model = SVC(kernel="linear").fit(X0, y0)
        Z = model.predict(np.c_[XX.ravel(), YY.ravel()])
        Z = Z.reshape(XX.shape)
        plt.contourf(XX, YY, Z, alpha=0.6)
        plt.scatter(X0[:, 0], X0[:, 1], marker='o', c=y0, s=40, 
                    linewidth=1, edgecolor='gray', alpha=0.7)
        plt.title(Counter(y0))
    else:
        plt.subplot(121)
        model = SVC(kernel="linear").fit(X0, y0)
        Z = model.predict(np.c_[XX.ravel(), YY.ravel()])
        Z = Z.reshape(XX.shape)
        plt.contourf(XX, YY, Z, alpha=0.6)
        plt.scatter(X0[:, 0], X0[:, 1], marker='o', c=y0, s=40, 
                    linewidth=1, edgecolor='gray', alpha=0.7)
        plt.xlim(-2, 4)
        plt.ylim(-3, 4)
        plt.title(Counter(y0))
        plt.subplot(122)
        model = SVC(kernel="linear").fit(X, y)
        Z = model.predict(np.c_[XX.ravel(), YY.ravel()])
        Z = Z.reshape(XX.shape)
        plt.contourf(XX, YY, Z, alpha=0.6)
        plt.scatter(X[:, 0], X[:, 1], marker='o', c=y, s=40, 
                    linewidth=1, edgecolor='gray', alpha=0.7)
        plt.xlim(-2, 4)
        plt.ylim(-3, 4)
        plt.title(Counter(y))
        plt.tight_layout()
    plt.show()
    
    return model.predict(X)
    
y_pred = plot_samples()

NameError: name 'X00' is not defined

In [17]:
from imblearn.combine import *

X, y = SMOTEENN(random_state=0).fit_sample(X_train, Y_train)
y_pred = plot_samples(X, y)

C:\Users\MK\AppData\Roaming\Python\Python36\site-packages\imblearn\utils\_validation.py:93: UserWarning: 'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got unknown instead.
  TARGET_KIND, type_of_target(y)))
C:\Users\MK\AppData\Roaming\Python\Python36\site-packages\imblearn\utils\_validation.py:93: UserWarning: 'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got unknown instead.
  TARGET_KIND, type_of_target(y)))
C:\Users\MK\AppData\Roaming\Python\Python36\site-packages\imblearn\utils\_validation.py:93: UserWarning: 'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got unknown instead.
  TARGET_KIND, type_of_target(y)))
C:\Users\MK\AppData\Roaming\Python\Python36\site-packages\imblearn\utils\_validation.py:93: UserWarning: 'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got unknown instead.
  TARGET_KIND, type_of_target(y)))


NameError: name 'plot_samples' is not defined

In [ ]:
X, y = RandomOverSampler(random_state=0).fit_sample(X0, y0)
y_pred = plot_samples(X, y)